<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/JAdventure-master/jad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53924 sha256=3797db6515c64e49d52d5bb32eb0aa44d7ea9d0023cdc5c4c74ddd5f663e4258
  Stored in directory: /root/.cache/pip/wheels/78/ae/2e/5ce789557ca59281a463dabe9d6522e429be240322148e5379
Successfully built docx


In [2]:
!pip install exceptions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement exceptions (from versions: none)
ERROR: No matching distribution found for exceptions


In [3]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
save_file_csv = 'JAdventure2.csv'
file_csv =  'JAdventure.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master' #Newly stored class address


In [6]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
#       print(root) #current directory path
#       print(files) #All non-directory subfiles in the current path
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [7]:
file_name(file_dir)

In [8]:
fileList

['/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/MenuItem.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/MonsterFactory.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/Menus.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/BattleMenu.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/ItemRepositoryTest.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/Storage.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/EntityTest.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/DeathException.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/Coordinate.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/MonsterTest.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/ItemTest.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/JAdv

In [11]:
# stopwords
stopwords = ['abstract', 'java','assert', 'boolean', 'break', 'byte', 'case', 'catch', 'class','char', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'int', 'interface', 'long', 'native', 'new', 'package','private','public', 'protected',  'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'transient', 'try', 'void', 'volatile', 'while']
dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}
# stopwords = {'abstract':'', 'java':'','assert':'', 'boolean':'', 'break':'', 'byte':'', 'case':'', 'catch':'', 'class':'','char':'', 'continue':'', 'default':'', 'do':'', 'double':'', 'else':'', 'enum':'', 'extends':'', 'final':'', 'finally':'', 'float':'', 'for':'', 'if':'', 'implements':'', 'import':'', 'int':'', 'interface':'', 'long':'', 'native':'', 'new':'', 'package':'','private':'','public':'', 'protected':'',  'return':'', 'short':'', 'static':'', 'strictfp':'', 'super':'', 'switch':'', 'synchronized':'', 'this':'', 'throw':'', 'transient':'', 'try':'','void':'', 'volatile':'', 'while':''}

In [12]:
def lm_find_unchinese(line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line) #排除汉字
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese) #排除中文符号
    #print("unchinese:",unchinese)
    return unchinese

In [13]:
def del_stopwords(line):
#     for line in file.readlines():
#     dicts = {i: '' for i in punctuation}
    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

#     for word in stopwords:
#           new_line = new_line.replace(word, '')

    return new_line


In [14]:
index = []
index_all = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        list = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name)
        name = filename[:-4]
        class_name.append(name)
        list.append(name)
        list2.append(name)
        
        for line in file.readlines():
            line = lm_find_unchinese(line)
            line = del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
#             if len(coefs) != 0:
#                 if coefs != ' ':
#                     if coefs != '\n':
            for i in coefs:
                list.append(i)

        index.append(list)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
            print('T')
        
#         print (list2)
        
        Brain_Class.append(list2)
            


T
T
T
T


In [15]:
for k in range(0,len(Brain_Class)):
    a = Brain_Class[k][0]
    if '_' in a:
        n = a.find('_')
        new_str = ''
        for i in range(0, n):
            new_str = new_str + a[i]
        Brain_Class[k][0] = new_str
    print(Brain_Class[k][0])

MenuItem
MonsterFactory
Menus
BattleMenu
ItemRepositoryTest
Storage
EntityTest
DeathException
Coordinate
MonsterTest
ItemTest
JAdventure
Player
Wolf
StorageTest
ChooseClassMenu
MainMenu
CommandParser
NpcRepository
Monster
CommandCollectionTest
ItemRepository
PlayerTest
LocationTest
Giant
Item
CoordinateTest
ItemStack
Bugbear
Trading
Steps
MonsterFactoryTest
BackpackDebugPrompt
CharacterChange
QueueProvider
RepositoryException
ConversationManagerTest
Skeleton
Entity
EncounteredNpcRepositoryTest
EncounteredNpcRepository
Location
GameBeans
LocationRepository
NPC
Game
Troll
ConversationManager
Client
LocationRepositoryTest
Goblin
NpcRepositoryTest
ILocation
Line


In [16]:
Brain_Class = np.array(Brain_Class)
Brain_Class

array([['MenuItem', 'False'],
       ['MonsterFactory', 'False'],
       ['Menus', 'False'],
       ['BattleMenu', 'True'],
       ['ItemRepositoryTest', 'False'],
       ['Storage', 'False'],
       ['EntityTest', 'False'],
       ['DeathException', 'False'],
       ['Coordinate', 'False'],
       ['MonsterTest', 'False'],
       ['ItemTest', 'False'],
       ['JAdventure', 'False'],
       ['Player', 'False'],
       ['Wolf', 'False'],
       ['StorageTest', 'False'],
       ['ChooseClassMenu', 'False'],
       ['MainMenu', 'True'],
       ['CommandParser', 'False'],
       ['NpcRepository', 'False'],
       ['Monster', 'False'],
       ['CommandCollectionTest', 'False'],
       ['ItemRepository', 'False'],
       ['PlayerTest', 'False'],
       ['LocationTest', 'False'],
       ['Giant', 'False'],
       ['Item', 'False'],
       ['CoordinateTest', 'False'],
       ['ItemStack', 'False'],
       ['Bugbear', 'False'],
       ['Trading', 'True'],
       ['Steps', 'False'],
       ['Mo

In [17]:
len(index)

54

In [18]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list

for item in index_all:
    # print(item)
    item = additem(item)
    # print(item)

In [19]:
news_df = np.array(index,dtype=object)
print(news_df)

[list(['MenuItem', 'SetString', 'return', 'String', 'return', 'String', 'return', 'MenuItemString', 'commandString', 'thisaltCommandsnew', 'MenuItemString', 'commandString', 'descriptionString', 'for', 'String', 'altCommand', 'altCommands', 'void', 'setAltCommandsSetString', 'void', 'setCommandString'])
 list(['MonsterFactory', 'Monster', 'generateMonsterPlayer', 'int', 'randomIntrandomnextInt5', 'if', 'randomInt', 'playergetLocationgetDangerRating', 'playergetLocationType', 'return', 'return', 'return', 'else', 'Monster', 'getCaveMonsterint', 'int', 'if', 'randomInt', '0', 'return', 'new', 'else', 'if', 'randomInt', '1', 'return', 'new', 'else', 'return', 'new', 'Monster', 'getForestMonsterint', 'int', 'if', 'randomInt', '0', 'return', 'new', 'else', 'if', 'randomInt', '1', 'return', 'new', 'else', 'return', 'new', 'Monster', 'getMountainMonsterint', 'int', 'if', 'randomInt', '0', 'return', 'new', 'else', 'if', 'randomInt', '1', 'return', 'new', 'else', 'if', 'randomInt', '2', 'return

In [20]:

def f_lsa(news_df, n):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(news_df)
 

    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)
    X2 = svd_model.fit_transform(X) 


    terms = vectorizer.get_feature_names()
   
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        # print("Topic " + str(i) + ": ")
        for t in sorted_terms:
            print(t[0])
    print(" ")

    D = svd_model.components_
    T = X2
    S = np.diagflat(svd_model.singular_values_)

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n
    # print("-------------acsm----------------------")
    # print(acsm)

    return acsm

In [21]:
q = 0
acsm= []
for i in index:
    lists = []
    print("*************",i,"****************")
    # print("------------------------------------------------------------------")
    for j in i:
        lists.append(j)
    # lists = deleteDuplicatedElementFromList(lists)
    if len(lists) > 1:
        acsm.append(f_lsa(lists,len(i)))
    else :
        acsm.append(1)
#

# print(type(class_name))
# print(type(acsm))

************* ['MenuItem', 'SetString', 'return', 'String', 'return', 'String', 'return', 'MenuItemString', 'commandString', 'thisaltCommandsnew', 'MenuItemString', 'commandString', 'descriptionString', 'for', 'String', 'altCommand', 'altCommands', 'void', 'setAltCommandsSetString', 'void', 'setCommandString'] ****************
string
thisaltcommandsnew
for
setstring
menuitemstring
commandstring
altcommand
 
************* ['MonsterFactory', 'Monster', 'generateMonsterPlayer', 'int', 'randomIntrandomnextInt5', 'if', 'randomInt', 'playergetLocationgetDangerRating', 'playergetLocationType', 'return', 'return', 'return', 'else', 'Monster', 'getCaveMonsterint', 'int', 'if', 'randomInt', '0', 'return', 'new', 'else', 'if', 'randomInt', '1', 'return', 'new', 'else', 'return', 'new', 'Monster', 'getForestMonsterint', 'int', 'if', 'randomInt', '0', 'return', 'new', 'else', 'if', 'randomInt', '1', 'return', 'new', 'else', 'return', 'new', 'Monster', 'getMountainMonsterint', 'int', 'if', 'randomIn

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


************* ['ItemRepositoryTest', 'public', 'void', 'ItemRepository', 'itemReponew', 'Item', 'assertEqualsMilk', 'in', 'a', 'public', 'void', 'ItemRepository', 'itemReponew', 'Item', 'assertEqualsA', 'nice', 'Item', 'assertEqualsMilk', 'in', 'a', 'Item', 'MapStringInteger', 'propertiesnew', 'return', 'new', 'Itemegg1foodeggA', 'nice', 'Item', 'MapStringInteger', 'propertiesnew', 'return', 'new', 'ItemmilkbottlepotionmilkMilk', 'in', 'a', 'public', 'void', 'ItemRepository', 'Item', 'Item', 'Item', 'Item', 'public', 'void', 'InputStream', 'ItemRepository', 'itemReponew', 'JsonReader', 'readernew', 'JsonReadernew', 'Item', 'public', 'void', 'String', 'inputitems', 'fegg1idfegg1type', 'foodnameeggdescriptionA', 'nice', 'egg', 'positionmouthlevel1', 'propertiesweight1value3health2', 'pmil1idpmil1type', 'potionnamemilkdescriptionMilk', 'in', 'a', 'bottle', 'positionmouthlevel1', 'propertiesweight1value10health5', 'JsonReader', 'jsonReadernew', 'JsonReadernew', 'ItemRepository', 'itemRepon

In [22]:
c = []
for i in range(len(class_name)):
      for j in range(len(acsm)):
        if i==j:
            t=[class_name[i],acsm[j]]
            c.append(t)

# c = list(zip(class_name,acsm))
# print(type(c))
c = np.array(c)
# print(c)
# print(c[0][0])

In [27]:
df =pd.read_csv("/content/drive/MyDrive/delesmell/dataset_class/JAdventure-master/JAdventure.csv") 

In [28]:
x= df.iloc[:, 0]

In [29]:
c

array([['MenuItem', '0.061262593561189925'],
       ['MonsterFactory', '0.43582128457836755'],
       ['Menus', '0.1855084802501575'],
       ['BattleMenu', '0.026407034070793048'],
       ['ItemRepositoryTest', '0.16288246044143184'],
       ['Storage', '0.204127051000311'],
       ['EntityTest', '0.2621440323170091'],
       ['DeathException', '0.06250000000000003'],
       ['Coordinate', '0.1763098543681195'],
       ['MonsterTest', '0.1631639260023956'],
       ['ItemTest', '0.32653061224489793'],
       ['JAdventure', '0.03976055584448669'],
       ['Player', '0.09332923950235854'],
       ['Wolf', '0.5289256198347108'],
       ['StorageTest', '0.13892634108614196'],
       ['ChooseClassMenu', '0.016206330997528162'],
       ['MainMenu', '0.07308660614450183'],
       ['CommandParser', '0.08000000000000004'],
       ['NpcRepository', '0.2970537805086381'],
       ['Monster', '0.0432530714747075'],
       ['CommandCollectionTest', '0.2777779808023541'],
       ['ItemRepository', '0

In [30]:
Brain_Class

array([['MenuItem', 'False'],
       ['MonsterFactory', 'False'],
       ['Menus', 'False'],
       ['BattleMenu', 'True'],
       ['ItemRepositoryTest', 'False'],
       ['Storage', 'False'],
       ['EntityTest', 'False'],
       ['DeathException', 'False'],
       ['Coordinate', 'False'],
       ['MonsterTest', 'False'],
       ['ItemTest', 'False'],
       ['JAdventure', 'False'],
       ['Player', 'False'],
       ['Wolf', 'False'],
       ['StorageTest', 'False'],
       ['ChooseClassMenu', 'False'],
       ['MainMenu', 'True'],
       ['CommandParser', 'False'],
       ['NpcRepository', 'False'],
       ['Monster', 'False'],
       ['CommandCollectionTest', 'False'],
       ['ItemRepository', 'False'],
       ['PlayerTest', 'False'],
       ['LocationTest', 'False'],
       ['Giant', 'False'],
       ['Item', 'False'],
       ['CoordinateTest', 'False'],
       ['ItemStack', 'False'],
       ['Bugbear', 'False'],
       ['Trading', 'True'],
       ['Steps', 'False'],
       ['Mo

In [31]:
for i in range(0,len(Brain_Class)):
    if 'False' == Brain_Class[i][1]:
        q = 1
    else :
        print('*')

*
*
*
*


In [32]:

ngList=[] 

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(c[i][0])
        st = st[1:]
#         print(st)
        if index == st:
            ngList.append(c[i][1])
            q = 0
            break
#             print('*')
    if q==1 :
        ngList.append(1)
        

In [33]:
brainList=[] 

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(Brain_Class[i][0])
        st = st[1:]
        if index == st:
            brainList.append(Brain_Class[i][1])
            q = 0
            break
    if q==1 :
        brainList.append('0')
        
        
print(len(brainList))
print(len(x))

40
40


In [34]:
column=df.columns.tolist()

In [35]:
df['acsm'] = ngList 
df['is_brain'] = brainList 
df.to_csv(save_file_csv,mode = 'a',index =False)

In [36]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOA,NOAM,NOD,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain
0,Steps,1.0,0,0,1,0,0,0,0,0,...,0,0,0,3,0,1.00,3,1.0,1,0
1,CharacterChange,5.0,0,0,1,0,0,0,0,0,...,0,0,0,2,0,0.00,10,1.0,1,0
2,GameBeans,1.0,0,0,1,0,0,0,0,0,...,0,0,0,4,0,0.00,4,1.0,1,0
3,JAdventure,5.0,3,0,1,2,0,0,0,0,...,1,0,0,3,0,0.33,15,1.0,1,0
4,EntityTest,1.0,1,0,1,1,0,0,0,0,...,1,0,0,17,0,0.67,17,1.0,1,0
